In [1]:
from datetime import datetime
import pandas as pd
from configparser import ConfigParser
from urllib.request import urlopen
import requests
import json
from bs4 import BeautifulSoup    

parser = ConfigParser()
parser.read("covid_app/settings.ini")

[]

In [84]:
class DataLoader:
    def __init__(self, parser: ConfigParser):
        
        self.latest_load = None
        self.data = None
    def load_data(self):
        # local, external

        def load_jhu():
            error_msg = "cannot load jhu data, no url provided"
            try:
                lookup_table = pd.read_csv(parser.get("urls", "jhu_lookup_url"))
                lookup_table.rename(
                    columns={"Country_Region": "region", "Long_": "Lon"}, inplace=True
                )
                return lookup_table
            except:
                print(error_msg)
                return None
            
            if lookup_table:
                def read_prepare_data(url):
                    try:
                        data_raw = pd.read_csv(parser.get("urlss", url))
                        data_raw.rename(columns={"Country/Region": "region"}, inplace=True)
                        data = (
                            data_raw.groupby("region")
                            .sum()
                            .drop(columns=["Lat", "Long"])
                            .reset_index()
                        )

                        return data
                    except:
                        print(error_msg)
                        return None
                
                def create_timeseries(data, lookup_table, value_name):
                    try:
                        
                        id_vars = "region"
                        var_name = "date"
                        timeseries = pd.melt(
                            data, id_vars=id_vars, var_name=var_name, value_name=value_name
                        )
                        timeseries = pd.merge(
                            lookup_table[["iso2", "iso3", "code3", "Lat", "Lon", id_vars]]
                            .groupby(id_vars)
                            .first(),
                            timeseries,
                            on=id_vars,
                            how="inner",
                        )
                        timeseries.loc[:, var_name] = pd.to_datetime(timeseries.loc[:, var_name])
                        return timeseries
                    except:
                        print(error_msg)
                        return None
                        

                confirmed_data = read_prepare_data("jhu_confirmed_url")
                deaths_data = read_prepare_data("jhu_deaths_url")
                recovered_data = read_prepare_data("jhu_recovered_url")

                confirmed = create_timeseries(confirmed_data, lookup, "confirmed")
                deaths = create_timeseries(deaths_data, lookup, "deaths")
                recovered = create_timeseries(recovered_data, lookup, "recovered")

                data = pd.merge(
                    deaths[["date", "region", "iso3", "Lat", "Lon", "deaths"]],
                    confirmed[["date", "confirmed", "iso3"]],
                    on=["iso3", "date"],
                    how="inner",
                )
                data = pd.merge(
                    data,
                    recovered[["date", "recovered", "iso3"]],
                    on=["iso3", "date"],
                    how="inner",
                )

                data.rename(columns={"confirmed": "cases"}, inplace=True)

                return data
                
            else:
                print(error)
                return None
            
                
                
        self.latest_load = datetime.now()
        self.data = load_jhu()
    def prepare(self):
        # join tables, create timeseries
        pass
    def write_data(self):
        # to csv
        if self.data is not None:
            self.data.to_csv("test.csv")
        else:
            print("nothing to write")

In [85]:
%%time
load = Loader(parser)
load.load_data()
load.data.head()

CPU times: user 19.5 ms, sys: 2.33 ms, total: 21.8 ms
Wall time: 65.9 ms


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,region,Lat,Lon,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [143]:
url = "https://raw.githubusercontent.com/ditori1976/covid_app/master/data/countries.csv"
load = Loader(url)
load.load_data()
print(load.latest_load)
print(load.data.head())
load.write_data()

2020-05-06 07:49:51.352032
   Rank           name      pop2019  pop2018  GrowthRate       area   Density
0     1          China  1433783.686      NaN      1.0039  9706961.0  147.7068
1     2          India  1366417.754      NaN      1.0099  3287590.0  415.6290
2     3  United States   329064.917      NaN      1.0059  9372610.0   35.1092
3     4      Indonesia   270625.568      NaN      1.0107  1904569.0  142.0928
4     5       Pakistan   216565.318      NaN      1.0200   881912.0  245.5634


In [144]:
load = Loader()
load.load_data()
load.write_data()

no url provided
nothing to write


In [145]:
class Data(Loader):
    def __init__(self):
        super().__init__(self)

In [147]:
data = Data()

print(data.latest_load) # latest_load not callable!

data.__dict__#.keys()


None


{'url': <__main__.Data at 0x7f82f3ee6588>, 'latest_load': None, 'data': None}